In [1]:
import numpy as np

In [2]:
#   .txt
file_name = 'academicMisconduct.txt'

### Deliverable 1


In [3]:
file = open(file_name, 'r')
content = file.read()
file.close()

num_char = len(content)

unique_char = set(content)
num_unique_char = len(unique_char)

# each character is encoded by the same number of bits
average_bit = np.round((np.log(num_unique_char)), 2)
total_bit = np.round(average_bit * num_char)

# print(content, end='\n\n')
# print(unique_char, end='\n\n')

print('Total number of characters: {} \nNumber of distinct characters: {} \nTotal number of bits needed to encode the text: {} \nAverage number of bits per character: {}'.format(
    num_char, num_unique_char, total_bit, average_bit))

Total number of characters: 9881 
Number of distinct characters: 65 
Total number of bits needed to encode the text: 41204.0 
Average number of bits per character: 4.17


### Deliverable 2


In [4]:
file = open(file_name, 'r')
content = file.read()
file.close()

num_char = len(content)

char_freq = dict()
for char in content:
    char_freq[char] = char_freq.get(char, 0) + 1

# sort unique characters by frequency in descending order
sorted_unique_chars = sorted(
    char_freq.keys(), key=lambda char: char_freq[char], reverse=True)

print(sorted_unique_chars)

total_bit = 0
i = 1
for char in sorted_unique_chars:
    total_bit += char_freq[char] * i
    i += 1

average_bit = np.round(total_bit / num_char, 2)

print('Total numer of bits: {} \nAverage number of bits: {}'.format(
    total_bit, average_bit))

[' ', 'e', 't', 'i', 'a', 'n', 'o', 's', 'r', 'd', 'c', 'u', 'h', 'l', 'm', 'f', 'p', 'g', '\t', '.', 'b', 'w', 'y', 'v', '\n', ',', 'A', 'k', 'C', 'x', ')', 'M', 'I', 'S', ':', 'P', 'U', '1', 'T', 'L', 'j', "'", 'R', '"', 'F', '3', '2', '(', 'G', '9', 'H', '4', 'E', '-', '6', 'W', 'D', '/', '5', '7', 'q', '8', '0', 'B', 'O']
Total numer of bits: 85373 
Average number of bits: 8.64


### Deliverable 3


In [5]:
class Node:
    def __init__(self, chars, freq, left=None, right=None):
        self.chars = chars
        self.freq = freq
        self.left = left
        self.right = right

In [6]:
def get_char_freq(file_name):
    file = open(file_name, 'r')
    content = file.read()
    file.close()

    char_freq = dict()
    for char in content:
        char_freq[char] = char_freq.get(char, 0) + 1

    return char_freq

In [7]:
def create_node(char_freq):
    nodes = list()

    for char, freq in char_freq.items():
        nodes.append(Node([char], freq))

    return nodes

In [8]:
def merge_trees(left_root, right_root):
    chars = list(set(left_root.chars + right_root.chars))
    freq = left_root.freq + right_root.freq

    return Node(chars, freq, left_root, right_root)

In [9]:
def create_binary_tree(nodes):
    sorted_nodes = sorted(nodes, key=lambda x: x.freq)

    root = sorted_nodes[0]
    for i in range(1, len(sorted_nodes), 1):
        root = merge_trees(sorted_nodes[i], root)

    return root

In [10]:
# HELPER
def visualise_tree(root, level=0):
    print(('\t' * level) + '{} - {} [{}]'.format(level, root.chars, root.freq))

    level += 1
    if root.left:
        visualise_tree(root.left, level)

    if root.right:
        visualise_tree(root.right, level)

In [11]:
test_case = {'a': 20, 'b': 2, 'c': 1}
root = create_binary_tree(create_node(test_case))

visualise_tree(root)

0 - ['b', 'c', 'a'] [23]
	1 - ['a'] [20]
	1 - ['b', 'c'] [3]
		2 - ['b'] [2]
		2 - ['c'] [1]


### Deliverable 4


In [12]:
def find_in_tree(char, root):
    level = 0
    while root:
        if not root.left:
            if char == root.chars[0]:
                return level
            else:
                break

        level += 1
        if char == root.left.chars[0]:
            return level

        root = root.right

    return None

In [13]:
find_in_tree('a', root)

1

In [14]:
find_in_tree('b', root)

2

In [15]:
find_in_tree('c', root)

2

In [16]:
find_in_tree('d', root)

### Deliverable 5


In [17]:
def tree_from_file(file_name):
    file = open(file_name, 'r')
    content = file.read()
    file.close()

    char_freq = dict()
    for char in content:
        char_freq[char] = char_freq.get(char, 0) + 1

    # sort unique characters by frequency in descending order
    sorted_unique_chars = sorted(
        char_freq.keys(), key=lambda char: char_freq[char], reverse=True)

    def create_binary_tree(sorted_unique_chars):
        nonlocal char_freq

        freq = sum(char_freq[char] for char in sorted_unique_chars)
        node = Node(sorted_unique_chars, freq)

        n = len(sorted_unique_chars)
        if n == 1:
            return node
        elif n == 2:
            node.left = Node([sorted_unique_chars[0]],
                             char_freq[sorted_unique_chars[0]])
            node.right = Node([sorted_unique_chars[1]],
                              char_freq[sorted_unique_chars[1]])
            return node

        left = 0
        right = freq

        for i in range(0, n, 1):
            left += char_freq[sorted_unique_chars[i]]
            right -= char_freq[sorted_unique_chars[i]]

            if left >= right:
                node.left = create_binary_tree(sorted_unique_chars[0:i])
                node.right = create_binary_tree(sorted_unique_chars[i:])
                return node

    node = create_binary_tree(sorted_unique_chars)

    def calculate_bit(node, level=0):
        if node.left is None and node.right is None:
            return node.freq * level
        else:
            level += 1
            return calculate_bit(node.left, level) + calculate_bit(node.right, level)

    total_bit = calculate_bit(node)

    num_char = len(content)
    average_bit = np.round(total_bit / num_char, 2)

    print('Total number of characters: {} \nNumber of distinct characters: {} \nTotal number of bits needed to encode the text: {} \nAverage number of bits per character: {}'.format(
        num_char, len(sorted_unique_chars), total_bit, average_bit))

    visualise_tree(node)

In [18]:
def tree_from_file_2(file_name):
    file = open(file_name, 'r')
    content = file.read()
    file.close()

    char_freq = dict()
    for char in content:
        char_freq[char] = char_freq.get(char, 0) + 1

    nodes = list()
    for char, freq in char_freq.items():
        nodes.append(Node([char], freq))

    while len(nodes) > 1:
        right_node = min(nodes, key=lambda node: node.freq)
        nodes.remove(right_node)

        left_node = min(nodes, key=lambda node: node.freq)
        nodes.remove(left_node)

        new_node = merge_trees(left_node, right_node)
        nodes.append(new_node)

    root = nodes[0]

    def calculate_bit(node, level=0):
        if node.left is None and node.right is None:
            return node.freq * level
        else:
            level += 1
            return calculate_bit(node.left, level) + calculate_bit(node.right, level)

    total_bit = calculate_bit(root)

    num_char = len(content)
    average_bit = np.round(total_bit / num_char, 2)

    print('Total number of characters: {} \nNumber of distinct characters: {} \nTotal number of bits needed to encode the text: {} \nAverage number of bits per character: {}'.format(
        num_char, len(sorted_unique_chars), total_bit, average_bit))

    visualise_tree(root)

In [19]:
tree_from_file(file_name)

Total number of characters: 9881 
Number of distinct characters: 65 
Total number of bits needed to encode the text: 45045 
Average number of bits per character: 4.56
0 - [' ', 'e', 't', 'i', 'a', 'n', 'o', 's', 'r', 'd', 'c', 'u', 'h', 'l', 'm', 'f', 'p', 'g', '\t', '.', 'b', 'w', 'y', 'v', '\n', ',', 'A', 'k', 'C', 'x', ')', 'M', 'I', 'S', ':', 'P', 'U', '1', 'T', 'L', 'j', "'", 'R', '"', 'F', '3', '2', '(', 'G', '9', 'H', '4', 'E', '-', '6', 'W', 'D', '/', '5', '7', 'q', '8', '0', 'B', 'O'] [9881]
	1 - [' ', 'e', 't', 'i', 'a'] [4417]
		2 - [' '] [1439]
		2 - ['e', 't', 'i', 'a'] [2978]
			3 - ['e'] [924]
			3 - ['t', 'i', 'a'] [2054]
				4 - ['t'] [790]
				4 - ['i', 'a'] [1264]
					5 - ['i'] [644]
					5 - ['a'] [620]
	1 - ['n', 'o', 's', 'r', 'd', 'c', 'u', 'h', 'l', 'm', 'f', 'p', 'g', '\t', '.', 'b', 'w', 'y', 'v', '\n', ',', 'A', 'k', 'C', 'x', ')', 'M', 'I', 'S', ':', 'P', 'U', '1', 'T', 'L', 'j', "'", 'R', '"', 'F', '3', '2', '(', 'G', '9', 'H', '4', 'E', '-', '6', 'W', 'D'

In [20]:
import time

start = time.time()
tree_from_file_2(file_name)
end = time.time()

print(end - start)

Total number of characters: 9881 
Number of distinct characters: 65 
Total number of bits needed to encode the text: 44218 
Average number of bits per character: 4.48
0 - ['E', 'e', '.', 'f', '9', '-', 'l', '7', 'a', 'D', 'F', '2', 'I', 'c', 'x', '\t', '\n', '4', '/', 'p', '3', 'y', 'j', 'G', 'O', 'h', 'U', 'P', 'B', 'u', '"', 's', 'T', ' ', ')', 'm', '6', '1', 'o', 'R', 'b', 'v', '8', 'd', 'i', 'W', 'C', 'q', ',', '5', ':', 'w', 'S', 'g', 'n', '(', 'A', 'M', 'L', '0', 'r', 'H', 'k', 't', "'"] [9881]
	1 - ['.', 'f', '-', 'a', 'D', '2', 'x', '\t', 'c', 'p', '3', 'y', 'G', 'U', 'P', 'T', ' ', ')', 'm', '1', 'b', 'd', 'i', 'W', 'C', ',', ':', 'w', 'g', '(', 'A', 'k', 't'] [5690]
		2 - ['.', '-', 'D', '2', '\t', '3', 'G', 'U', 'P', 'T', ' ', 'm', '1', 'b', 'W', 'C', ',', ':', 'w', '(', 'k', 't'] [3058]
			3 - ['.', '-', 'D', '2', '\t', '3', 'G', 'U', 'P', 'T', 'm', '1', 'b', 'W', 'C', ',', ':', 'w', '(', 'k', 't'] [1619]
				4 - ['.', '-', 'D', '2', '\t', '3', 'G', 'U', 'P', 'T', 'm', '1',